In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nest-competition/eligibilities.txt
/kaggle/input/nest-competition/facilities_cleaned.csv
/kaggle/input/nest-competition/eligibilities_cleaned.csv
/kaggle/input/nest-competition/drop_withdrawals_cleaned.csv
/kaggle/input/nest-competition/reported_events.txt
/kaggle/input/nest-competition/facilities.txt
/kaggle/input/nest-competition/usecase_2_.csv
/kaggle/input/nest-competition/drop_withdrawals.txt


In [2]:
import seaborn as sb
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/nest-competition/usecase_2_.csv')
df = df.rename(columns={"NCT Number": "nct_id"})

In [4]:
eligibilities = pd.read_csv('/kaggle/input/nest-competition/eligibilities.txt', sep='|')
eligibilities.head()

,id,nct_id,sampling_method,gender,minimum_age,maximum_age,healthy_volunteers,population,criteria,gender_description,gender_based,adult,child,older_adult
0,6260055,NCT05050916,NaN,FEMALE,19 Years,40 Years,f,NaN,* INCLUSION CRITERIA:~In order to be eligible ...,NaN,NaN,t,f,f
1,6260056,NCT01092156,NaN,FEMALE,18 Years,NaN,t,NaN,Inclusion Criteria:~* Pregnant women who inten...,NaN,NaN,t,f,t
2,6260057,NCT01218256,NaN,ALL,30 Years,80 Years,f,NaN,Inclusion Criteria:~* type 2 diabetes mellitus...,NaN,NaN,t,f,t
3,6260058,NCT03240432,NaN,ALL,60 Years,NaN,t,NaN,Inclusion Criteria:~To be eligible for the stu...,NaN,NaN,t,f,t
4,6260059,NCT04348578,NaN,ALL,18 Years,65 Years,t,NaN,"Inclusion Criteria:~* necrotic, single root te...",NaN,NaN,t,f,t


In [5]:
eligibilities.shape

(502301, 14)

In [6]:
eligibilities = eligibilities.merge(df[['nct_id']], on='nct_id', how='inner')
print(eligibilities.shape)

(68705, 14)


In [7]:
df.shape

(68960, 26)

In [8]:
eligibilities.drop_duplicates(keep="first", inplace=True)
eligibilities.shape

(68705, 14)

In [9]:
eligibilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68705 entries, 0 to 68704
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  68705 non-null  int64 
 1   nct_id              68705 non-null  object
 2   sampling_method     4 non-null      object
 3   gender              68667 non-null  object
 4   minimum_age         66209 non-null  object
 5   maximum_age         41573 non-null  object
 6   healthy_volunteers  68693 non-null  object
 7   population          4 non-null      object
 8   criteria            68704 non-null  object
 9   gender_description  2393 non-null   object
 10  gender_based        3598 non-null   object
 11  adult               68705 non-null  object
 12  child               68705 non-null  object
 13  older_adult         68705 non-null  object
dtypes: int64(1), object(13)
memory usage: 7.3+ MB


In [10]:
# Insert 'population' and 'gender_description' before '~Exclusion Criteria'
def insert_values(row):
    # Initialize insertion string
    insertion = ""
    
    # Check if 'population' is not NaN
    if pd.notna(row['population']):
        insertion += f'~*{row["population"]} '
    
    # Check if 'gender_description' is not NaN
    if pd.notna(row['gender_description']):
        insertion += f'~*{row["gender_description"]} '

    # If insertion string is not empty, add it before '~Exclusion Criteria'
    if insertion:
        return row['criteria'].replace('~Exclusion Criteria', f'{insertion}~Exclusion Criteria', 1)
    
    # Return original criteria if nothing to insert
    return row['criteria']

# Apply the function row-wise
eligibilities['criteria'] = eligibilities.apply(insert_values, axis=1)

# Drop the 'population' and 'gender_description' columns
eligibilities = eligibilities.drop(['population', 'gender_description'], axis='columns')

# Display updated DataFrame
eligibilities.head()

,id,nct_id,sampling_method,gender,minimum_age,maximum_age,healthy_volunteers,criteria,gender_based,adult,child,older_adult
0,6260055,NCT05050916,NaN,FEMALE,19 Years,40 Years,f,* INCLUSION CRITERIA:~In order to be eligible ...,NaN,t,f,f
1,6260058,NCT03240432,NaN,ALL,60 Years,NaN,t,Inclusion Criteria:~To be eligible for the stu...,NaN,t,f,t
2,6260059,NCT04348578,NaN,ALL,18 Years,65 Years,t,"Inclusion Criteria:~* necrotic, single root te...",NaN,t,f,t
3,6260067,NCT06144983,NaN,ALL,18 Years,NaN,t,Inclusion Criteria:~1. Absence of any history ...,t,t,f,t
4,6361029,NCT03191318,NaN,ALL,18 Years,65 Years,f,Inclusion Criteria:~* All patients undergoing ...,NaN,t,f,t


In [11]:
eligibilities['sampling_method'].unique()

array([nan, 'NON_PROBABILITY_SAMPLE', 'PROBABILITY_SAMPLE'], dtype=object)

In [12]:
eligibilities = eligibilities.drop(['sampling_method'], axis='columns')
eligibilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68705 entries, 0 to 68704
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  68705 non-null  int64 
 1   nct_id              68705 non-null  object
 2   gender              68667 non-null  object
 3   minimum_age         66209 non-null  object
 4   maximum_age         41573 non-null  object
 5   healthy_volunteers  68693 non-null  object
 6   criteria            68704 non-null  object
 7   gender_based        3598 non-null   object
 8   adult               68705 non-null  object
 9   child               68705 non-null  object
 10  older_adult         68705 non-null  object
dtypes: int64(1), object(10)
memory usage: 5.8+ MB


In [13]:
eligibilities['gender_based'].unique()

array([nan, 't'], dtype=object)

In [14]:
eligibilities['gender_based'].fillna('f', inplace=True)

/tmp/ipykernel_24/3003346701.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eligibilities['gender_based'].fillna('f', inplace=True)


In [15]:
eligibilities['gender'].unique()

array(['FEMALE', 'ALL', 'MALE', nan], dtype=object)

In [16]:
eligibilities['gender'].value_counts(dropna=False)

gender
ALL       58529
FEMALE     6789
MALE       3349
NaN          38
Name: count, dtype: int64

In [17]:
eligibilities['gender'].fillna('ALL', inplace=True)

/tmp/ipykernel_24/2059911870.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eligibilities['gender'].fillna('ALL', inplace=True)


In [18]:
eligibilities['minimum_age'].unique()

array(['19 Years', '60 Years', '18 Years', '50 Years', '13 Years',
       '8 Years', '1 Hour', '16 Years', '40 Years', '30 Years', '6 Years',
       '35 Years', '4 Years', '0 Years', '20 Years', '3 Days', '1 Year',
       '21 Years', '9 Years', '2 Years', '36 Years', '14 Years',
       '5 Years', nan, '17 Years', '15 Years', '3 Years', '65 Years',
       '10 Years', '25 Years', '7 Years', '11 Years', '3 Weeks',
       '3 Months', '22 Years', '12 Years', '18 Months', '1 Day',
       '0 Hours', '8 Months', '6 Months', '45 Years', '41 Years',
       '75 Years', '2 Months', '32 Weeks', '125 Months', '0 Days',
       '51 Years', '55 Years', '224 Days', '70 Years', '1 Month',
       '0 Weeks', '12 Months', '24 Weeks', '2 Hours', '26 Years',
       '23 Years', '80 Years', '66 Years', '62 Years', '34 Weeks',
       '6 Hours', '42 Days', '24 Hours', '1 Minute', '32 Years',
       '22 Weeks', '60 Months', '2 Days', '60 Days', '56 Days',
       '28 Years', '65 Months', '38 Years', '48 Hours', '9 

In [19]:
eligibilities['minimum_age'].isna().sum()

2496

In [20]:
eligibilities['minimum_age'].fillna('0 Minutes', inplace=True)

/tmp/ipykernel_24/1966530466.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eligibilities['minimum_age'].fillna('0 Minutes', inplace=True)


In [21]:
eligibilities['maximum_age'].isna().sum()

27132

In [22]:
eligibilities['maximum_age'].unique()

array(['40 Years', nan, '65 Years', '66 Years', '45 Years', '80 Years',
       '72 Years', '55 Years', '21 Years', '48 Hours', '70 Years',
       '25 Years', '60 Years', '95 Years', '100 Years', '18 Years',
       '75 Years', '7 Years', '49 Years', '28 Days', '5 Years',
       '42 Years', '99 Years', '50 Years', '17 Years', '74 Years',
       '64 Years', '35 Years', '36 Years', '90 Years', '85 Years',
       '30 Years', '12 Years', '6 Years', '11 Years', '59 Years',
       '6 Months', '54 Years', '44 Years', '84 Years', '62 Years',
       '24 Years', '19 Years', '29 Days', '14 Years', '41 Weeks',
       '19 Months', '48 Months', '56 Years', '92 Days', '22 Years',
       '1 Week', '14 Days', '69 Years', '47 Months', '32 Years',
       '16 Years', '41 Years', '81 Years', '9 Years', '4 Months',
       '39 Years', '42 Weeks', '200 Months', '5 Days', '34 Years',
       '33 Years', '78 Months', '18 Months', '79 Years', '258 Days',
       '60 Minutes', '105 Years', '72 Months', '83 Years', '1

In [23]:
eligibilities['maximum_age'].fillna('200 Years', inplace=True)

/tmp/ipykernel_24/18217775.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eligibilities['maximum_age'].fillna('200 Years', inplace=True)


In [24]:
eligibilities['healthy_volunteers'].unique()

array(['f', 't', nan], dtype=object)

In [25]:
eligibilities['healthy_volunteers'].value_counts(dropna=False)

healthy_volunteers
f      42103
t      26590
NaN       12
Name: count, dtype: int64

In [26]:
eligibilities['healthy_volunteers'].fillna('f', inplace=True)

/tmp/ipykernel_24/1692022156.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eligibilities['healthy_volunteers'].fillna('f', inplace=True)


In [27]:
eligibilities.head()

,id,nct_id,gender,minimum_age,maximum_age,healthy_volunteers,criteria,gender_based,adult,child,older_adult
0,6260055,NCT05050916,FEMALE,19 Years,40 Years,f,* INCLUSION CRITERIA:~In order to be eligible ...,f,t,f,f
1,6260058,NCT03240432,ALL,60 Years,200 Years,t,Inclusion Criteria:~To be eligible for the stu...,f,t,f,t
2,6260059,NCT04348578,ALL,18 Years,65 Years,t,"Inclusion Criteria:~* necrotic, single root te...",f,t,f,t
3,6260067,NCT06144983,ALL,18 Years,200 Years,t,Inclusion Criteria:~1. Absence of any history ...,t,t,f,t
4,6361029,NCT03191318,ALL,18 Years,65 Years,f,Inclusion Criteria:~* All patients undergoing ...,f,t,f,t


In [28]:
eligibilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68705 entries, 0 to 68704
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  68705 non-null  int64 
 1   nct_id              68705 non-null  object
 2   gender              68705 non-null  object
 3   minimum_age         68705 non-null  object
 4   maximum_age         68705 non-null  object
 5   healthy_volunteers  68705 non-null  object
 6   criteria            68704 non-null  object
 7   gender_based        68705 non-null  object
 8   adult               68705 non-null  object
 9   child               68705 non-null  object
 10  older_adult         68705 non-null  object
dtypes: int64(1), object(10)
memory usage: 5.8+ MB


In [29]:
# Save the DataFrame as a CSV file
eligibilities.to_csv('eligibilities_cleaned.csv', index=False)

# Kaggle specific: Generate a download link
from IPython.display import FileLink
FileLink('eligibilities_cleaned.csv')

/kaggle/working/eligibilities_cleaned.csv